In [14]:
import os
import sys
sys.path.append('../libraries/')
from match_behavior import match_behavior
from moviepy.editor import VideoFileClip, concatenate_videoclips, clips_array, vfx

## Load neural data

In [15]:
data_dir = '../processed_data'
model_dir = '../hmm_models'
behavior_matcher = match_behavior(data_dir, model_dir)

In [16]:
neural_file_name = 'm1_fr0.1hz_30hz_0s_7200s_0.1s_bins_nooverlap.sav'
behavior_matcher.load_data(neural_file_name, sample_rate=10)

Spike times are binned into shape for hmm: (36000, 46)


## Load behaviors

In [17]:
behavior_file_name = 'ag25290_day2_iter13'
behavior_names = ['insignificant', 'insignificant', 'insignificant', 'insignificant', 'rear',
         'insignificant', 'insignificant', 'investigate type 1', 'investigate type 2', 'contra-itch',
         'investigate type 3', 'insignificant', 'insignificant', 'insignificant', 'contra-body groom',
         'face groom type 1', 'dive/scrunch', 'head groom', 'ipsi-orient', 'insignificant',
         'face groom type 2', 'ipsi-body groom', 'ipsi-itch type 1', 'insignificant', 'insignificant', 
         'paw groom', 'locomotion', 'insignificant', 'contra-orient']
behavior_matcher.load_behavior(behavior_file_name, behavior_names, frame_rate=60, delay=6.8503)

mouse stayed in nest for 0.28108749470484795% of time
File #0 (a 6 body parts by 984391 frames) has 30 classes


## Load hmm

In [18]:
behavior_matcher.load_hmm(16, 'merged')

--------------------------------------------------
A 16-state hmm is used to match with behaviors.
--------------------------------------------------


## Load video

In [19]:
video_dir = os.path.join(data_dir, 'video')
video_file_name = '2021-11-23_13-19-58_1h.mp4'
# Get 1h clip
clip = VideoFileClip(os.path.join(video_dir, video_file_name))
# Set frame rate
clip.set_fps(behavior_matcher.frame_rate)
print(f'The 1st hour video clip loaded.')

The 1st hour video clip loaded.


## Match behaviors identified for hmm states with video

In [23]:
def hmm_defined_behavior_to_video(clip, behavior_matcher, lead_seconds, lag_seconds):
    hmm_states = behavior_matcher.hmm_states_on_data()
    _, idt_behav_for_states_names = behavior_matcher.hmm_state_to_behavior(lead_seconds, lag_seconds)

    sample_rate = behavior_matcher.sample_rate
    behavior_counter = {}
    for i, hmm_state in enumerate(np.unique(hmm_states)):
        behavior_name = idt_behav_for_states_names[i]
        if behavior_name in behavior_counter:
            behavior_counter[behavior_name] = behavior_counter[behavior_name] + 1
        else:
            behavior_counter.update({behavior_name: 0})

        print(f'Preparing video clip for {behavior_name}_{behavior_counter[behavior_name]}...')
        hmm_state_idcs = np.where(hmm_states == hmm_state)[0]
        hmm_state_clip = concatenate_videoclips([clip.subclip(idx/sample_rate, idx/sample_rate + 1/sample_rate) for idx in hmm_state_idcs])
        hmm_state_clip.set_fps(behavior_matcher.frame_rate)
        print('Video clip prepared.')

        session_dir = os.path.join(video_dir, f'lead_{lead_seconds}_s_lag_{lag_seconds}_s')
        if not os.path.exists(session_dir):
            os.mkdir(session_dir)
        behavior_dir = os.path.join(session_dir, behavior_name)
        if not os.path.exists(behavior_dir):
            os.mkdir(behavior_dir)
        print(f'Writing video clip to folder {behavior_dir}...')
        hmm_state_clip.write_videofile(os.path.join(behavior_dir, f'hmm_state_{hmm_state}_{behavior_name}_{behavior_counter[behavior_name]}_lead_{lead_seconds}_s_lag_{lag_seconds}_s.mp4'),
                                        codec='mpeg4',
                                        threads='20', bitrate='32768k', # 8K resolution
                                        ffmpeg_params=[
                                            '-tile-columns', '6', '-frame-parallel', '0',
                                            '-auto-alt-ref', '1', '-lag-in-frames', '25', '-g',
                                            '128', '-pix_fmt', 'yuv420p', '-row-mt', '1'])
        print(f'hmm_state_{hmm_state}_{behavior_name}_{behavior_counter[behavior_name]}_lead_{lead_seconds}_s_lag_{lag_seconds}_s.mp4 is written to {behavior_dir}.')

In [ ]:
hmm_defined_behavior_to_video(clip, behavior_matcher, lead_seconds=0.1, lag_seconds=0.3)

In [ ]:
hmm_defined_behavior_to_video(clip, behavior_matcher, lead_seconds=0, lag_seconds=0.3)

## Display locomotions altogether in folder lead_0.1_s_lag_0.3_s/

In [ ]:
# Total 7 locomotions
lead_1_lag_3_dir = os.path.join(video_dir, 'lead_0.1_s_lag_0.3_s')
behav_dir = os.path.join(lead_1_lag_3_dir, 'locomotion')
file_list = os.listdir(behav_dir)
file_list.append('background_filler.mp4')
file_list.append('background_filler.mp4')

# Make up to 9 slots for the sake of formatting
rows, cols = (3, 3)
clips_arr = [[0]*cols for _ in range(rows)]
for i, file_name in enumerate(file_list):
    try:
        clip = VideoFileClip(os.path.join(behav_dir, file_name)).subclip(0, 60)
    except:
        clip = VideoFileClip(os.path.join(video_dir, file_name)).subclip(0, 60)
    clip.set_fps(60)
    clip.fx(vfx.speedx, 0.125)
    clips_arr[i//cols][i%cols] = clip

collect_clip = clips_array(clips_arr)
collect_clip.write_videofile(os.path.join(lead_1_lag_3_dir, 'locomotion_collect.mp4'),
                                        codec='mpeg4',
                                        threads='20', bitrate='32768k', # 8K resolution
                                        ffmpeg_params=[
                                            '-tile-columns', '6', '-frame-parallel', '0',
                                            '-auto-alt-ref', '1', '-lag-in-frames', '25', '-g',
                                            '128', '-pix_fmt', 'yuv420p', '-row-mt', '1'])


## Display investigate type 2s altogether in folder lead_0_s_lag_0.3_s/

In [105]:
# Total 9 investigate type 2s
lead_0_lag_3_dir = os.path.join(video_dir, 'lead_0_s_lag_0.3_s')
behav_dir = os.path.join(lead_0_lag_3_dir, 'investigate type 2')
file_list = os.listdir(behav_dir)

# Make up to 9 slots for the sake of formatting
rows, cols = (3, 3)
clips_arr = [[0]*cols for _ in range(rows)]
for i, file_name in enumerate(file_list):
    try:
        clip = VideoFileClip(os.path.join(behav_dir, file_name)).subclip(0, 60)
    except:
        clip = VideoFileClip(os.path.join(video_dir, file_name)).subclip(0, 60)
    clip.set_fps(60)
    clip.fx(vfx.speedx, 0.125)
    clips_arr[i//cols][i%cols] = clip

collect_clip = clips_array(clips_arr)
collect_clip.write_videofile(os.path.join(lead_0_lag_3_dir, 'investigate type 2_collect.mp4'),
                                        codec='mpeg4',
                                        threads='20', bitrate='32768k', # 8K resolution
                                        ffmpeg_params=[
                                            '-tile-columns', '6', '-frame-parallel', '0',
                                            '-auto-alt-ref', '1', '-lag-in-frames', '25', '-g',
                                            '128', '-pix_fmt', 'yuv420p', '-row-mt', '1'])

Moviepy - Building video ../processed_data/video/investigate type 2_collect.mp4.
MoviePy - Writing audio in investigate type 2_collectTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ../processed_data/video/investigate type 2_collect.mp4



Moviepy - Done !
Moviepy - video ready ../processed_data/video/investigate type 2_collect.mp4
